In [1]:
from google.cloud import bigquery
import pandas as pd
import google
import os

# Establece la variable de entorno GOOGLE_APPLICATION_CREDENTIALS con la ruta al archivo de credenciales
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:/Users/raul.raggio/AppData/Roaming/gcloud/application_default_credentials.json"
credentials, project = google.auth.default()

project="peya-chile"


client = bigquery.Client(project=project, credentials=credentials)
print("Client creating using default project: {}".format(client.project))

Client creating using default project: peya-chile


c:\Users\raul.raggio\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\auth\_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [4]:
flag=1

In [5]:

if flag ==0:
    
    sql1 = """

WITH 
 log_rider_orders as (
   select
   country_code,
     o.entity.id as entity_id
     , o.vendor.vendor_code AS vendor_code
     , o.created_date
     , o.created_at
     , o.platform_order_code as order_code
     , o.timings.vendor_late
     , order_status
     , o.estimated_prep_buffer
     , o.timings.rider_late
     , o.timings.estimated_walk_in_duration
     ,o.timings.assumed_actual_preparation_time
     ,o.timings.at_vendor_time_cleaned
     ,o.is_preorder
     ,o.sent_to_vendor_at
     , auto_transition.pickup as auto_transition
     , d.rider_near_restaurant_at
     , DATETIME_ADD(d.rider_near_restaurant_at,INTERVAL CAST((60+COALESCE(o.timings.estimated_walk_in_duration,0)) AS int) SECOND) as rider_at_restaurant
     , original_scheduled_pickup_at
     , requested_pickup_at AS adjsuted_original_scheduled_pickup --> vendor communicated it will take longer
     , COALESCE(requested_pickup_at,original_scheduled_pickup_at) AS scheduled_pickup_at
     , food_is_ready_at
     , rider_picked_up_at
     ,round((o.timings.avoidable_wait_time/60),2) as AWT_minutes
     ,dim_partner.main_cousine_category_name as cuisine,
    dim_partner.franchise.franchise_name as franchise_name,
    dim_partner.business_type.business_type_name,
    dim_partner.partner_name  AS dim_partner_partner_name,
    dim_partner.partner_id  AS dim_partner_partner_id,
    dim_partner.city.name  AS city_name,
    dim_partner.ordersReceptionSystem.name as ReceptionSystem,
      ifnull(v.pos.integration_name,"no_integration") as integration 

   FROM `fulfillment-dwh-production.curated_data_shared.orders` o

   LEFT JOIN `peya-bi-tools-pro.il_core.dim_partner` AS dim_partner ON dim_partner.partner_id=SAFE_CAST(o.vendor.vendor_code AS INT)
   LEFT JOIN  `peya-data-origins-pro.cl_core.growth_vendors` gv on dim_partner.partner_id= safe_cast (vendor_code as integer)
LEFT JOIN UNNEST(gv.vendor) AS v

   CROSS JOIN unnest(deliveries) AS d ON is_primary

   WHERE 1=1
   and country_code="cl"
     AND o.created_date >='2025-01-01'
 )

 select *

 from log_rider_orders

    """
    data1=client.query(sql1).to_dataframe()
    data1.to_csv("data1.csv")
else:
    
    data1=pd.read_csv("data1.csv")

C:\Users\raul.raggio\AppData\Local\Temp\ipykernel_29264\1850703132.py:63: DtypeWarning: Columns (3,6) have mixed types. Specify dtype option on import or set low_memory=False.
  data1=pd.read_csv("data1.csv")


In [6]:
data1

,Unnamed: 0,country_code,entity_id,vendor_code,created_date,created_at,order_code,vendor_late,order_status,estimated_prep_buffer,...,rider_picked_up_at,AWT_minutes,cuisine,franchise_name,business_type_name,dim_partner_partner_name,dim_partner_partner_id,city_name,ReceptionSystem,integration
0,0,cl,PY_CL,177612,2025-01-18,2025-01-18 02:05:22.272177+00:00,1423218217,0.0,completed,0,...,2025-01-18 02:10:13.293124+00:00,0.00,NaN,Market PedidosYa,Market,PedidosYa Market Temuco,177612.0,Temuco,Shopper RPS,no_integration
1,1,cl,PY_CL,177612,2025-01-18,2025-01-18 01:12:21.407756+00:00,1423144884,232.0,completed,0,...,2025-01-18 01:23:14.628285+00:00,3.87,NaN,Market PedidosYa,Market,PedidosYa Market Temuco,177612.0,Temuco,Shopper RPS,no_integration
2,2,cl,PY_CL,177612,2025-01-18,2025-01-18 17:33:55.825993+00:00,1423722588,0.0,completed,0,...,2025-01-18 17:36:10.552443+00:00,0.00,NaN,Market PedidosYa,Market,PedidosYa Market Temuco,177612.0,Temuco,Shopper RPS,no_integration
3,3,cl,PY_CL,177612,2025-01-18,2025-01-18 18:58:01.406077+00:00,1423806147,0.0,completed,0,...,2025-01-18 19:03:27.878806+00:00,0.00,NaN,Market PedidosYa,Market,PedidosYa Market Temuco,177612.0,Temuco,Shopper RPS,no_integration
4,4,cl,PY_CL,177612,2025-01-18,2025-01-18 20:43:17.180282+00:00,1423874140,0.0,completed,0,...,2025-01-18 20:49:08.359849+00:00,0.00,NaN,Market PedidosYa,Market,PedidosYa Market Temuco,177612.0,Temuco,Shopper RPS,no_integration
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4761988,4761988,cl,PY_CL,453417,2025-02-01,2025-02-01 02:15:48.524954+00:00,1437699472,924.0,completed,60,...,2025-02-01 02:56:22.185975+00:00,NaN,Sushi,NaN,Restaurant,Apetitoso - Almirante Latorre,453417.0,Los Ángeles,9 Cookies,no_integration
4761989,4761989,cl,PY_CL,501690,2025-02-01,2025-02-01 18:45:20.055548+00:00,1438159717,0.0,completed,120,...,2025-02-01 19:06:11.890354+00:00,0.00,Hamburguesas,NaN,Restaurant,Big Family Burger 164,501690.0,Los Ángeles,9 Cookies,no_integration
4761990,4761990,cl,PY_CL,201294,2025-02-01,2025-02-01 16:36:05.997542+00:00,1437977903,771.0,completed,60,...,2025-02-01 17:01:13.993302+00:00,12.85,Comida Asiática,NaN,Restaurant,Bowls Mania,201294.0,Los Ángeles,9 Cookies,no_integration
4761991,4761991,cl,PY_CL,201294,2025-02-01,2025-02-01 18:48:44.638411+00:00,1438160255,184.0,completed,60,...,2025-02-01 19:05:07.960091+00:00,2.98,Comida Asiática,NaN,Restaurant,Bowls Mania,201294.0,Los Ángeles,9 Cookies,no_integration


In [7]:
df1=data1

In [8]:
df1.columns

Index(['Unnamed: 0', 'country_code', 'entity_id', 'vendor_code',
       'created_date', 'created_at', 'order_code', 'vendor_late',
       'order_status', 'estimated_prep_buffer', 'rider_late',
       'estimated_walk_in_duration', 'assumed_actual_preparation_time',
       'at_vendor_time_cleaned', 'is_preorder', 'sent_to_vendor_at',
       'auto_transition', 'rider_near_restaurant_at', 'rider_at_restaurant',
       'original_scheduled_pickup_at', 'adjsuted_original_scheduled_pickup',
       'scheduled_pickup_at', 'food_is_ready_at', 'rider_picked_up_at',
       'AWT_minutes', 'cuisine', 'franchise_name', 'business_type_name',
       'dim_partner_partner_name', 'dim_partner_partner_id', 'city_name',
       'ReceptionSystem', 'integration'],
      dtype='object')

In [9]:
df1=data1
df1['created_at'] = pd.to_datetime(df1['created_at'], errors='coerce')
df1['franchise_name'] = df1['franchise_name'].str.lower()
df1['cuisine'].fillna("no-cuisine", inplace=True)
df1['franchise_name'].fillna("no franchise", inplace=True)

df1['hour_of_day'] = df1['created_at'].dt.hour
df1['day_of_week'] = df1['created_at'].dt.day_name()
#dffoods=df1[(df1["business_type_name"]=="Restaurant") | (df1["business_type_name"]=="Coffee")]


df1['created_date'] = pd.to_datetime(df1['created_date'])

df1["month"]=df1["created_date"].dt.month



C:\Users\raul.raggio\AppData\Local\Temp\ipykernel_29264\3352326291.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df1['cuisine'].fillna("no-cuisine", inplace=True)
C:\Users\raul.raggio\AppData\Local\Temp\ipykernel_29264\3352326291.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

In [10]:
# Convertir las columnas de fechas a la zona horaria de Chile (CL)
df1["vendor_late"]=df1["vendor_late"]/60
df1['created_at'] = df1['created_at'].dt.tz_convert('America/Santiago')
df1['sent_to_vendor_at'] = pd.to_datetime(df1['sent_to_vendor_at'], format='mixed').dt.tz_convert('America/Santiago')
df1['rider_near_restaurant_at'] = pd.to_datetime(df1['rider_near_restaurant_at'], format='mixed').dt.tz_convert('America/Santiago')
df1['rider_at_restaurant'] = pd.to_datetime(df1['rider_at_restaurant'], format='mixed').dt.tz_convert('America/Santiago')
df1['original_scheduled_pickup_at'] = pd.to_datetime(df1['original_scheduled_pickup_at'], format='mixed').dt.tz_convert('America/Santiago')
df1['adjsuted_original_scheduled_pickup'] = pd.to_datetime(df1['adjsuted_original_scheduled_pickup'], format='mixed').dt.tz_convert('America/Santiago')
df1['scheduled_pickup_at'] = pd.to_datetime(df1['scheduled_pickup_at'], format='mixed').dt.tz_convert('America/Santiago')
df1['food_is_ready_at'] = pd.to_datetime(df1['food_is_ready_at'], format='mixed').dt.tz_convert('America/Santiago')
df1['rider_picked_up_at'] = pd.to_datetime(df1['rider_picked_up_at'], format='mixed').dt.tz_convert('America/Santiago')


df1['is_vendor_late_10'] = df1['vendor_late'].apply(lambda x: 1 if x >= 10 else 0)
df1['is_vendor_late_nn'] = df1['vendor_late'].apply(lambda x: 1 if pd.notnull(x) else 0)

In [11]:
df1['created_date'] = df1['created_at'].dt.date
df1["created_date"] = pd.to_datetime(df1["created_date"], errors='coerce').dt.date

In [12]:
df1

,Unnamed: 0,country_code,entity_id,vendor_code,created_date,created_at,order_code,vendor_late,order_status,estimated_prep_buffer,...,dim_partner_partner_name,dim_partner_partner_id,city_name,ReceptionSystem,integration,hour_of_day,day_of_week,month,is_vendor_late_10,is_vendor_late_nn
0,0,cl,PY_CL,177612,2025-01-17,2025-01-17 23:05:22.272177-03:00,1423218217,0.000000,completed,0,...,PedidosYa Market Temuco,177612.0,Temuco,Shopper RPS,no_integration,2.0,Saturday,1,0,1
1,1,cl,PY_CL,177612,2025-01-17,2025-01-17 22:12:21.407756-03:00,1423144884,3.866667,completed,0,...,PedidosYa Market Temuco,177612.0,Temuco,Shopper RPS,no_integration,1.0,Saturday,1,0,1
2,2,cl,PY_CL,177612,2025-01-18,2025-01-18 14:33:55.825993-03:00,1423722588,0.000000,completed,0,...,PedidosYa Market Temuco,177612.0,Temuco,Shopper RPS,no_integration,17.0,Saturday,1,0,1
3,3,cl,PY_CL,177612,2025-01-18,2025-01-18 15:58:01.406077-03:00,1423806147,0.000000,completed,0,...,PedidosYa Market Temuco,177612.0,Temuco,Shopper RPS,no_integration,18.0,Saturday,1,0,1
4,4,cl,PY_CL,177612,2025-01-18,2025-01-18 17:43:17.180282-03:00,1423874140,0.000000,completed,0,...,PedidosYa Market Temuco,177612.0,Temuco,Shopper RPS,no_integration,20.0,Saturday,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4761988,4761988,cl,PY_CL,453417,2025-01-31,2025-01-31 23:15:48.524954-03:00,1437699472,15.400000,completed,60,...,Apetitoso - Almirante Latorre,453417.0,Los Ángeles,9 Cookies,no_integration,2.0,Saturday,2,1,1
4761989,4761989,cl,PY_CL,501690,2025-02-01,2025-02-01 15:45:20.055548-03:00,1438159717,0.000000,completed,120,...,Big Family Burger 164,501690.0,Los Ángeles,9 Cookies,no_integration,18.0,Saturday,2,0,1
4761990,4761990,cl,PY_CL,201294,2025-02-01,2025-02-01 13:36:05.997542-03:00,1437977903,12.850000,completed,60,...,Bowls Mania,201294.0,Los Ángeles,9 Cookies,no_integration,16.0,Saturday,2,1,1
4761991,4761991,cl,PY_CL,201294,2025-02-01,2025-02-01 15:48:44.638411-03:00,1438160255,3.066667,completed,60,...,Bowls Mania,201294.0,Los Ángeles,9 Cookies,no_integration,18.0,Saturday,2,0,1


In [13]:


def asignar_horario(hora):
    if hora<=7:
        return '0-7'
    elif hora<=11:
        return '8-11'
    elif hora<=14:
        return '12-14'
    elif hora<=17:
        return '15-17'
    elif hora<=20:
        return '18-20'
    else:
        return '21-23'
    
def asignar_dia(dia):
    if dia in ["Friday","Saturday","Sunday"]:
        return 'FIN_DE_SEMANA'

    else:
        return 'DIA_DE_SEMANA'

df1['horario'] = df1["hour_of_day"].apply(asignar_horario)
df1['dia_semana'] = df1["day_of_week"].apply(asignar_dia)

In [14]:
df1.columns

Index(['Unnamed: 0', 'country_code', 'entity_id', 'vendor_code',
       'created_date', 'created_at', 'order_code', 'vendor_late',
       'order_status', 'estimated_prep_buffer', 'rider_late',
       'estimated_walk_in_duration', 'assumed_actual_preparation_time',
       'at_vendor_time_cleaned', 'is_preorder', 'sent_to_vendor_at',
       'auto_transition', 'rider_near_restaurant_at', 'rider_at_restaurant',
       'original_scheduled_pickup_at', 'adjsuted_original_scheduled_pickup',
       'scheduled_pickup_at', 'food_is_ready_at', 'rider_picked_up_at',
       'AWT_minutes', 'cuisine', 'franchise_name', 'business_type_name',
       'dim_partner_partner_name', 'dim_partner_partner_id', 'city_name',
       'ReceptionSystem', 'integration', 'hour_of_day', 'day_of_week', 'month',
       'is_vendor_late_10', 'is_vendor_late_nn', 'horario', 'dia_semana'],
      dtype='object')

In [15]:
df1

,Unnamed: 0,country_code,entity_id,vendor_code,created_date,created_at,order_code,vendor_late,order_status,estimated_prep_buffer,...,city_name,ReceptionSystem,integration,hour_of_day,day_of_week,month,is_vendor_late_10,is_vendor_late_nn,horario,dia_semana
0,0,cl,PY_CL,177612,2025-01-17,2025-01-17 23:05:22.272177-03:00,1423218217,0.000000,completed,0,...,Temuco,Shopper RPS,no_integration,2.0,Saturday,1,0,1,0-7,FIN_DE_SEMANA
1,1,cl,PY_CL,177612,2025-01-17,2025-01-17 22:12:21.407756-03:00,1423144884,3.866667,completed,0,...,Temuco,Shopper RPS,no_integration,1.0,Saturday,1,0,1,0-7,FIN_DE_SEMANA
2,2,cl,PY_CL,177612,2025-01-18,2025-01-18 14:33:55.825993-03:00,1423722588,0.000000,completed,0,...,Temuco,Shopper RPS,no_integration,17.0,Saturday,1,0,1,15-17,FIN_DE_SEMANA
3,3,cl,PY_CL,177612,2025-01-18,2025-01-18 15:58:01.406077-03:00,1423806147,0.000000,completed,0,...,Temuco,Shopper RPS,no_integration,18.0,Saturday,1,0,1,18-20,FIN_DE_SEMANA
4,4,cl,PY_CL,177612,2025-01-18,2025-01-18 17:43:17.180282-03:00,1423874140,0.000000,completed,0,...,Temuco,Shopper RPS,no_integration,20.0,Saturday,1,0,1,18-20,FIN_DE_SEMANA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4761988,4761988,cl,PY_CL,453417,2025-01-31,2025-01-31 23:15:48.524954-03:00,1437699472,15.400000,completed,60,...,Los Ángeles,9 Cookies,no_integration,2.0,Saturday,2,1,1,0-7,FIN_DE_SEMANA
4761989,4761989,cl,PY_CL,501690,2025-02-01,2025-02-01 15:45:20.055548-03:00,1438159717,0.000000,completed,120,...,Los Ángeles,9 Cookies,no_integration,18.0,Saturday,2,0,1,18-20,FIN_DE_SEMANA
4761990,4761990,cl,PY_CL,201294,2025-02-01,2025-02-01 13:36:05.997542-03:00,1437977903,12.850000,completed,60,...,Los Ángeles,9 Cookies,no_integration,16.0,Saturday,2,1,1,15-17,FIN_DE_SEMANA
4761991,4761991,cl,PY_CL,201294,2025-02-01,2025-02-01 15:48:44.638411-03:00,1438160255,3.066667,completed,60,...,Los Ángeles,9 Cookies,no_integration,18.0,Saturday,2,0,1,18-20,FIN_DE_SEMANA


In [16]:
df1['ept_adjusted'] = df1['adjsuted_original_scheduled_pickup'].notnull().astype(int)
df1['diff_minutes_ajusted'] = round((df1['adjsuted_original_scheduled_pickup'] - df1['original_scheduled_pickup_at']).dt.total_seconds() / 60)

In [26]:
df1["created_date"] = pd.to_datetime(df1["created_date"], errors='coerce')
df1["created_date"].max()

Timestamp('2025-02-25 00:00:00')

In [24]:
df1[ (df1['created_date'] == '2025-02-14')]

,Unnamed: 0,country_code,entity_id,vendor_code,created_date,created_at,order_code,vendor_late,order_status,estimated_prep_buffer,...,hour_of_day,day_of_week,month,is_vendor_late_10,is_vendor_late_nn,horario,dia_semana,ept_adjusted,diff_minutes_ajusted,grupo


In [27]:
import numpy as np
# Define las condiciones y asigna los valores a la columna 'grupo'
conditions = [
      (df1['created_date'] == '2025-02-14') ,
    #(df1['logistic_orders_created_date_local_date']== '2025-01-17	') ,
    (df1['created_date']== '2025-02-21') 
    #(df1['logistic_orders_created_date_local_date'] <= '2024-05-06')
]

values = ['14 feb', 'comparacion']

# Por defecto, si no cumple ninguna de las condiciones anteriores, se asigna 'otras_fechas'
df1['grupo'] = pd.Series(np.select(conditions, values, default='otras_fechas'))

In [28]:
dfaux=df1[df1["grupo"].isin(["14 feb","comparacion"])]
dfgrouped=dfaux.groupby(["grupo"]).agg({"order_code":"count","ept_adjusted":"sum","diff_minutes_ajusted":"mean"}).reset_index()
dfgrouped["%ept_adjusted"]=dfgrouped["ept_adjusted"]/dfgrouped["order_code"]
dfgrouped

,grupo,order_code,ept_adjusted,diff_minutes_ajusted,%ept_adjusted
0,14 feb,116397,1974,9.033435,0.016959
1,comparacion,98674,1446,8.662517,0.014654


In [32]:
dfaux=df1
dfgrouped=dfaux.groupby(["grupo","diff_minutes_ajusted"]).agg({"ept_adjusted":"sum"}).reset_index()
dfgrouped_pivot=dfgrouped.pivot(index="diff_minutes_ajusted",columns="grupo",values="ept_adjusted").reset_index()
dfgrouped_pivot["%_14 feb"]=dfgrouped_pivot["14 feb"]/dfgrouped_pivot["14 feb"].sum()
dfgrouped_pivot["%_comparacion"]=dfgrouped_pivot["comparacion"]/dfgrouped_pivot["comparacion"].sum()
dfgrouped_pivot

grupo,diff_minutes_ajusted,14 feb,comparacion,otras_fechas,%_14 feb,%_comparacion
0,-10.0,10.0,6.0,731.0,0.005066,0.004149
1,-8.0,8.0,6.0,175.0,0.004053,0.004149
2,-6.0,8.0,3.0,244.0,0.004053,0.002075
3,-5.0,NaN,NaN,1.0,NaN,NaN
4,-4.0,11.0,14.0,348.0,0.005572,0.009682
5,-2.0,4.0,14.0,247.0,0.002026,0.009682
6,2.0,32.0,39.0,1713.0,0.016211,0.026971
7,4.0,62.0,54.0,2479.0,0.031408,0.037344
8,5.0,NaN,NaN,1.0,NaN,NaN
9,6.0,105.0,112.0,3845.0,0.053191,0.077455
